# PCA applied to real data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from astropy.io import fits
from tqdm import tqdm

## Let's run the PCA over the 4 x 4 degree cube. We also consider the frequency binning.

In [ ]:
def PCA_eig_decomposition(cube):
    """
    Compute the eigenvectors and eigenvalue decomposition necessary for the 
    Principal Component Analysis.

    :param cube: input data cube. It can be reduced with a mask or not.
    """
    # Make each slide to have mean zero for both the signal and the full simulations
    mean_cube_2d =  np.mean(cube, axis=1)
    cube -= mean_cube_2d[:, np.newaxis]
    # Compute the covariance matrix
    cov = np.cov(cube) # (Nfreqs x Nfreqs)
    # Do eigendecomposition of covariance matrix
    eigvals, eigvecs = np.linalg.eig(cov)
    # Sort by eigenvalue
    idxs = np.argsort(eigvals)[::-1] # reverse order (biggest eigenvalue first)
    eigvals = eigvals[idxs]
    eigvecs = eigvecs[:,idxs]
    return cube, mean_cube_2d, eigvals, eigvecs

def proyect_PCA(nmodes, eigvecs, cube2d, mean_cube2d):
    """
    Function that removes the foregrounds using the PCA decomposition. It removes the first nmodes 
    obtained from the PCA decomposition.

    :param nmodes: number of modes to subtract
    :param eigvecs: eigenvectors from the PCA decomposition.
    :param cube2d: data cube with dimensions nsamples x Npix**2.
    :param mean_cube2d: mean of each slice of cube2d.
    """
    
    # Construct foreground filter operator by keeping only nmodes eigenmodes
    U_fg = eigvecs[:,:nmodes] # (Nfreqs, Nmodes)
    
    # Calculate foreground amplitudes for each line of sight
    fg_amps = np.dot(U_fg.T, cube2d) # (Nmodes, Npix**2)
    
    # Construct FG field and subtract from input data
    fg_field = np.dot(U_fg, fg_amps) + mean_cube2d[:, np.newaxis] # Operator times amplitudes + mean
    shape1 = (nmodes, int(np.sqrt(cube2d.shape[1])), int(np.sqrt(cube2d.shape[1])))
    shape2 = (int(cube2d.shape[0]), int(np.sqrt(cube2d.shape[1])), int(np.sqrt(cube2d.shape[1])))
    return fg_amps.reshape(shape1), fg_field.reshape(shape2)

## Load the data cube

The full frequency range of 90 MHz is divided into 15 MHz intervals, and for each a power spectrum is computed. To limit noise, only the central 4x4 degrees out of the full 8x8 degrees FoV are meant to be used for the power spectrum computation.

In [ ]:
fname_msn = '/mnt/sdc3a.MS.0/sdc3dataset/Image/ZW3.msn_image.fits'
fname_msw = '/mnt/sdc3a.MS.0/sdc3dataset/Image/ZW3.msw_image.fits'
data_shape = fits.open(fname_msw)[0].data.shape # shape of the data cube
header = fits.open(fname_msw)[0].header
angle = 4 # in degrees
resolution = 16 # in arcsecs
Npix_ps = angle*3600/resolution # number of pixels in the output image
Npix_data = data_shape[1] # number of pixels in the x-axis in the data
ind1 = int(Npix_data/2 - Npix_ps/2)
ind2 = int(Npix_data/2 + Npix_ps/2)
freq = np.arange(106, 196 + 0.1, 0.1)
freq_int = np.arange(0, 900 + 0.1, 150, dtype=int)
freq_int[len(freq_int)-1] += 1
data_ps = []
for i in tqdm(range(len(freq_int)-1)):
    data_ps.append(fits.open(fname_msw)[0].data[freq_int[i]:freq_int[i+1], ind1:ind2, ind1:ind2])

## Load the point sources mask

The file named "predictions.fit" contains not binary values of 0 or 1, but the probabilities produced by the model's output. These probabilistic masks are processed based on a predetermined threshold. For example, if a threshold of 0.4 is set, pixels with probabilities above this value will be marked as '1', and the rest as '0'.

We generate one foreground mask per frequency bin. This is done by calculating the maximum of the pixels in the slices in the frequency interval of interest.

In [ ]:
fname_mask = '/mnt/scratch/ps_masks/predictions.fit'
mask = fits.open(fname_mask)[0].data[:, ind1:ind2, ind1:ind2, 0]
threshold = 0.3 
mask_thresh = np.zeros_like(mask, dtype=int)
mask_thresh[mask > threshold] = 1
mask_binned = []
mask_neg = []
for i in tqdm(range(len(freq_int)-1)):
    # We create a common mask for all the slices.
    mask_binned.append(np.maximum.reduce(mask_thresh[freq_int[i]:freq_int[i+1], :, :], axis=0))
    mask_neg.append(np.zeros_like(mask_binned[i]))
    mask_neg[i][mask_binned[i] == 0] = 1

## Load the PFS and reduce its shape in order to match the dimension of the data and point sources mask

In [ ]:
fname_station_beam = '/mnt/sdc3a.MS.0/sdc3dataset/Image/station_beam.fits'
data_station_beam_shape = fits.open(fname_station_beam)[0].data.shape
Npix_beam = data_station_beam_shape[1]
index1_beam = int(Npix_beam/2 - Npix_ps/2)
index2_beam = int(Npix_beam/2 + Npix_ps/2)
beam = []
for i in tqdm(range(len(freq_int)-1)):
    beam.append(fits.open(fname_station_beam)[0].data[freq_int[i]:freq_int[i+1], index1_beam:index2_beam, index1_beam:index2_beam])

## We will run PCA following this steps:

    1) Run PCA on ncomponents = 4 (from which simulations have shown to be ideal), to get the projection matrix.

$$d\rightarrow P = WW^T.$$

    2) Project the PCA solution back to the image space from the components space.

$$d^{PCA} = P\cdot d.$$

    3) Obtain the clean solution from substraction of the PCA result to the data.
$$d^{clean} = d-d^{PCA}.$$

Products that I generated from the PCA cleaned solution.

    1) Cleaned PCA solution.

    2) Cleaned and smoothed PCA solution.

    3) Cleaned, masked and smoothed solution.

I do not do PSF deconvolution in any of the data products.

In [ ]:
nfreq_bins = (len(freq_int)-1)
ncomp = 4
cube_2d = [None] * nfreq_bins
mean_cube_2d = [None] * nfreq_bins
eigvals = [None] * nfreq_bins
eigvecs = [None] * nfreq_bins
fg_amps = [None] * nfreq_bins
fg_field = [None] * nfreq_bins
clean_field = [None] * nfreq_bins
clean_field_masked = [None] * nfreq_bins
for i in tqdm(range(len(freq_int)-1)):
    nsamples, nx, ny = data_ps[i].shape
    cube_2d[i] = data_ps[i].reshape((nsamples,nx*ny))
    cube_2d[i], mean_cube_2d[i], eigvals[i], eigvecs[i] = PCA_eig_decomposition(np.array(cube_2d[i]))
    fg_amps[i], fg_field[i] = proyect_PCA(ncomp, eigvecs[i], cube_2d[i], mean_cube_2d[i])
    clean_field[i] = data_ps[i] - fg_field[i]
    clean_field_masked[i] = clean_field[i] * mask_neg[i]

In [ ]:
plt.plot()
for i in tqdm(range(len(freq_int)-1)):
    plt.plot(np.cumsum(eigvals[i])/np.sum(eigvals[i]), label=f'Bin = {i}')
plt.xlabel('N components')
plt.ylabel('Cumulative explained variance')
plt.semilogx()
plt.ylim([0.99980, 1.00001])
plt.legend()
plt.show()

In [ ]:
slice = 50
from pylab import figure, cm
from matplotlib.colors import LogNorm
import scipy.ndimage as ndimage
# Define the number of rows and columns
nrows = 6
ncols = 4
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20,25))
# Iterate over rows
for i in range(nrows):
    im0 = axes[i, 0].imshow(np.log10(np.abs(data_ps[i][slice, :, :]))*mask_neg[i])
    plt.colorbar(im0, ax=axes[i, 0], orientation='vertical')
    im1 = axes[i, 1].imshow(np.log10(np.abs(fg_field[i][slice, :, :]))*mask_neg[i])
    plt.colorbar(im1, ax=axes[i, 1], orientation='vertical')
    im2 = axes[i, 2].imshow(clean_field[i][slice, :, :])
    plt.colorbar(im2, ax=axes[i, 2], orientation='vertical')
    im3 = axes[i, 3].imshow(ndimage.gaussian_filter(clean_field[i][slice, :, :], sigma=5, order=0))
    plt.colorbar(im3, ax=axes[i, 3], orientation='vertical')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
clean_field_smooth = [None] * nfreq_bins
clean_field_masked_smooth = [None] * nfreq_bins
for bin in tqdm(range(len(freq_int)-1)):
    clean_field_smooth[bin] = np.zeros_like(clean_field[bin])
    clean_field_masked_smooth[bin] = np.zeros_like(clean_field[bin])
    for slice in range(clean_field[bin].shape[0]):
        clean_field_smooth[bin][slice, :, :] = ndimage.gaussian_filter(clean_field[bin][slice, :, :], sigma=5, order=0)
        clean_field_masked_smooth[bin][slice, :, :] = ndimage.gaussian_filter(clean_field_masked[bin][slice, :, :], sigma=5, order=0)

In [ ]:
slice = 50
from pylab import figure, cm
from matplotlib.colors import LogNorm
import scipy.ndimage as ndimage
# Define the number of rows and columns
nrows = 6
ncols = 3
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20,25))
# Iterate over rows
for i in range(nrows):
    im0 = axes[i, 0].imshow(clean_field[bin][slice, :, :])
    plt.colorbar(im0, ax=axes[i, 0], orientation='vertical')
    im1 = axes[i, 1].imshow(clean_field_smooth[i][slice, :, :])
    plt.colorbar(im1, ax=axes[i, 1], orientation='vertical')
    im2 = axes[i, 2].imshow(clean_field_masked_smooth[i][slice, :, :])
    plt.colorbar(im2, ax=axes[i, 2], orientation='vertical')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

## We save our data products in several fits files 

In [ ]:
import os
os.makedirs('data_products', exist_ok=True)

for bin in tqdm(range(len(freq_int)-1)):
    fname = f'data_products/PCA_cleaned_{freq_int[bin]}_{freq_int[bin+1]}.fits'
    fname_smooth = f'data_products/PCA_cleaned_smooth_{freq_int[bin]}_{freq_int[bin+1]}.fits'
    fname_smooth_masked = f'data_products/PCA_cleaned_smooth_mask_{freq_int[bin]}_{freq_int[bin+1]}.fits'

    # Create a FITS header by modifying the one from the data fits file
    header['NAXIS1'] = clean_field[bin].shape[2]
    header['NAXIS2'] = clean_field[bin].shape[1]
    header['NAXIS3'] = clean_field[bin].shape[0]
    header['CRVAL3'] = freq[freq_int[bin]]*1e6 # in Hz
    # Create a FITS HDU (Header/Data Unit) containing the matrix
    hdu_clean = fits.PrimaryHDU(data=clean_field[bin], header=header)
    hdu_clean_smooth = fits.PrimaryHDU(data=clean_field_smooth[bin], header=header)
    hdu_clean_smooth_mask = fits.PrimaryHDU(data=clean_field_masked_smooth[bin], header=header)
    # Create a HDU list and add the primary HDU to it
    hdul_clean = fits.HDUList([hdu_clean])
    hdul_clean_smooth = fits.HDUList([hdu_clean_smooth])
    hdul_clean_smooth_mask = fits.HDUList([hdu_clean_smooth])
    
    # Write the HDU list to the FITS file
    hdul_clean.writeto(fname, overwrite=True)
    hdul_clean_smooth.writeto(fname_smooth, overwrite=True)
    hdul_clean_smooth_mask.writeto(fname_smooth_masked, overwrite=True)